# ESGF holdings summaries

The goal here is to summarize the holdings on ESGF nodes (LLNL only for now) to see exactly what model-scenario-variable combinations have data and which don't. We will assume we can treat variants interchangeably. Using the holdings table(s) generate with `esgf_holding.py`, any of the `grid_type`, `version`, `n_files`, or `filenmaes` can be used to determine if files were found for a given combination of attributes (NaN indicates no files found).

We will focus on daily data first, because this is the most useful for our purposes. It is often the case that there is monthly data if daily data exists for a certain combination, and monthly dat acan be generated from it if not.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

## LLNL Node

We only have the LLNL node summarized so far:

In [2]:
holdings_fps = ["llnl_esgf_holdings.csv", "llnl_esgf_holdings_e3sm.csv"]
holdings = pd.concat(
    (pd.read_csv(f) for f in holdings_fps),
    ignore_index=True,
)

## Daily data

These are the daily table Ids we are interested in so far:

In [3]:
day_freqs = ["day", "Eday", "Oday", "SIday"]
day_holdings = holdings.query("table_id in @day_freqs").copy()

Use the `n_files` column to determine if a given combination was found:

In [4]:
day_holdings["valid"] = day_holdings.n_files.notnull()

Groupy by the combinations and summarize:

In [5]:
tmp = (
    day_holdings.groupby(["model", "scenario", "variable"])
    .valid.sum()
    .reset_index(name="valid")
)
# this just swaps in checkmark for True
tmp["valid"] = ["\u2713" if x > 0 else "" for x in tmp["valid"]]
tmp

,model,scenario,variable,valid
0,ACCESS-CM2,historical,clt,✓
1,ACCESS-CM2,historical,evspsbl,
2,ACCESS-CM2,historical,hfls,✓
3,ACCESS-CM2,historical,hfss,✓
4,ACCESS-CM2,historical,hus,✓
...,...,...,...,...
2555,TaiESM1,ssp585,ts,
2556,TaiESM1,ssp585,ua,✓
2557,TaiESM1,ssp585,uas,
2558,TaiESM1,ssp585,va,✓


Unexpected, but length of df does not match length of all possible combinations. This is probably because if no data was found for any variable for a given scenario, it was omitted. 

In [6]:
print(
    "combos:",
    len(tmp.variable.unique()) * len(tmp.model.unique()) * len(tmp.scenario.unique()),
)
print("Number of rows:", len(tmp))

combos: 3040
Number of rows: 2560


In [7]:
tmp = tmp.pivot_table(
    values="valid",
    columns="variable",
    index=["model", "scenario"],
    aggfunc=lambda x: "".join(x),
)
tmp

variable                   clt evspsbl hfls hfss hus huss mrro mrsol mrsos pr  \
model           scenario                                                        
ACCESS-CM2      historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
CESM2           historical   ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CESM2-WACCM     historical   ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓              ✓   
CNRM-CM6-1-HR   historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓                       
                ssp370       ✓            ✓    ✓   ✓    ✓                       
                ssp585       ✓            ✓    ✓   ✓    ✓                   ✓   
E3SM-1-0        historical                              ✓                   ✓   
                ssp585                                  ✓                   ✓   
E3SM-1-1        historical                                                      
                ssp245                                                          
E3SM-1-1-ECA    historical                                                      
                ssp585                                  ✓                   ✓   
E3SM-2-0        historical                              ✓                   ✓   
                ssp370                                  ✓                   ✓   
EC-Earth3-Veg   historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓                   ✓   
GFDL-ESM4       historical                              ✓    ✓           ✓  ✓   
                ssp126                                  ✓    ✓           ✓  ✓   
                ssp245                                  ✓    ✓           ✓  ✓   
                ssp370                                  ✓    ✓           ✓  ✓   
                ssp585                                  ✓    ✓           ✓  ✓   
HadGEM3-GC31-LL historical   ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓          ✓        ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓          ✓        ✓   
HadGEM3-GC31-MM historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
KACE-1-0-G      historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp245       ✓ 

(This is just a double check that we have the right summary logic)

In [8]:
for (model, scenario), row in tmp.iterrows():
    for i in row.index:
        assert any(
            day_holdings.query(
                "model == @model & scenario == @scenario & variable == @i"
            ).valid
        ) == bool(row[i])

Do the same for monthly data:

In [9]:
mon_freqs = ["Amon", "Emon", "Lmon", "Omon", "LImon", "SImon"]
mon_holdings = holdings.query("table_id in @mon_freqs").copy()
mon_holdings["valid"] = mon_holdings.n_files.notnull()
tmp = (
    mon_holdings.groupby(["model", "scenario", "variable"])
    .valid.sum()
    .reset_index(name="valid")
)
tmp["valid"] = ["\u2713" if x > 0 else "" for x in tmp["valid"]]
tmp.pivot_table(
    values="valid",
    columns="variable",
    index=["model", "scenario"],
    aggfunc=lambda x: "".join(x),
)

variable                   clt evspsbl hfls hfss hus huss mrro mrsol mrsos pr  \
model           scenario                                                        
ACCESS-CM2      historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CESM2           historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CESM2-WACCM     historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CNRM-CM6-1-HR   historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
E3SM-1-0        historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
E3SM-1-1        historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp245       ✓                     ✓    ✓                   ✓   
E3SM-1-1-ECA    historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
E3SM-2-0        historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
EC-Earth3-Veg   historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
GFDL-ESM4       historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
HadGEM3-GC31-LL historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
HadGEM3-GC31-MM historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓              ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓              ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓              ✓   
KACE-1-0-G      historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓ 

## SNAP / ACDN holdings

Here we can do the same for the data we actually have.

### Daily

In [10]:
cmip6_dir = Path("/beegfs/CMIP6/arctic-cmip6/CMIP6")
all_fps = list(cmip6_dir.glob("**/*.nc"))


def get_size_on_disk(files):
    """lil function to get size on disk for a set of files (in TB)"""
    sizes = []
    for fp in files:
        sizes.append(fp.stat().st_size)
    return sum(sizes) / 1e12


dir_level_lu = dict(
    version=0, grid_type=1, var_id=2, table_id=3, variant=4, scenario=5, model=6
)


def get_fp_attrs(fp, attr):
    return fp.parents[dir_level_lu[attr]].name


rows = []
attr_list = ["model", "scenario", "table_id", "var_id"]
for fp in all_fps:
    rows.append({attr_name: get_fp_attrs(fp, attr_name) for attr_name in attr_list})
    rows[-1]["fp"] = fp

df = pd.DataFrame(rows)
df = df.drop_duplicates().reindex()
df["valid"] = "\u2713"


df.query("table_id in @day_freqs").pivot_table(
    values="valid",
    columns="var_id",
    index=["model", "scenario"],
    aggfunc=lambda x: np.unique(x)[0],
).fillna(
    ""
)  # .to_csv("snap_day_holdings.csv") # for saving to update google sheet

var_id                     clt evspsbl hfls hfss hus huss mrro mrsol mrsos pr  \
model           scenario                                                        
CESM2           historical   ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CESM2-WACCM     historical   ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓              ✓   
CNRM-CM6-1-HR   historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓                       
                ssp370       ✓            ✓    ✓   ✓    ✓                       
                ssp585       ✓            ✓    ✓   ✓    ✓                   ✓   
EC-Earth3-Veg   historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓                   ✓   
GFDL-ESM4       historical                              ✓    ✓           ✓  ✓   
                ssp126                                  ✓    ✓           ✓  ✓   
                ssp245                                  ✓    ✓           ✓  ✓   
                ssp370                                  ✓    ✓           ✓  ✓   
                ssp585                                  ✓    ✓           ✓  ✓   
HadGEM3-GC31-LL historical   ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓          ✓        ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓          ✓        ✓   
HadGEM3-GC31-MM historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓                   ✓   
KACE-1-0-G      historical                ✓    ✓   ✓         ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓                   ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓    ✓              ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓              ✓   
MIROC6          historical   ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp245       ✓            ✓    ✓   ✓         ✓           ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓           ✓  ✓   
MPI-ESM1-2-HR   historical   ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓            ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
MPI-ESM1-2-LR   historical   ✓ 

### Monthly


In [11]:
df.query("table_id in @mon_freqs").pivot_table(
    values="valid",
    columns="var_id",
    index=["model", "scenario"],
    aggfunc=lambda x: np.unique(x)[0],
).fillna(
    ""
)  # .to_csv("snap_day_holdings.csv")

var_id                     clt evspsbl hfls hfss hus huss mrro mrsol mrsos pr  \
model           scenario                                                        
CESM2           historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CESM2-WACCM     historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
CNRM-CM6-1-HR   historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓           ✓  ✓   
E3SM-1-1        historical   ✓                 ✓        ✓                       
                ssp245       ✓                          ✓                       
E3SM-2-0        historical   ✓                 ✓   ✓    ✓                       
                ssp370       ✓                 ✓   ✓    ✓                       
EC-Earth3-Veg   historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
GFDL-ESM4       historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
HadGEM3-GC31-LL historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓        ✓   
HadGEM3-GC31-MM historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓              ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓              ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓              ✓   
KACE-1-0-G      historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
MIROC6          historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp245       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp370       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp585       ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
MPI-ESM1-2-HR   historical   ✓       ✓    ✓    ✓   ✓    ✓    ✓     ✓     ✓  ✓   
                ssp126       ✓ 